##### Copyright 2020 Google Inc.

Licensed under the Apache License, Version 2.0 (the "License").
<!--
    Licensed to the Apache Software Foundation (ASF) under one
    or more contributor license agreements.  See the NOTICE file
    distributed with this work for additional information
    regarding copyright ownership.  The ASF licenses this file
    to you under the Apache License, Version 2.0 (the
    "License"); you may not use this file except in compliance
    with the License.  You may obtain a copy of the License at

      http://www.apache.org/licenses/LICENSE-2.0

    Unless required by applicable law or agreed to in writing,
    software distributed under the License is distributed on an
    "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY
    KIND, either express or implied.  See the License for the
    specific language governing permissions and limitations
    under the License.
-->


# Streaming

These notebooks shows two examples of streaming pipelines, one using the `InteractiveRunner` and one using the `DataflowRunner`.

Before getting into the code, let's prepare the environment.

In [ ]:
import logging
import json
import time
import traceback

import apache_beam as beam
from apache_beam.options.pipeline_options import PipelineOptions
from apache_beam.options import pipeline_options
from apache_beam.io.gcp.pubsub import ReadFromPubSub
from apache_beam.io.gcp.bigquery import BigQueryDisposition, WriteToBigQuery
from apache_beam.io import WriteToText

from apache_beam.runners.interactive.interactive_runner import InteractiveRunner
import apache_beam.runners.interactive.interactive_beam as ib
from apache_beam.runners import DataflowRunner

import google.auth

from utils.utils import publish_to_topic
from IPython.core.display import display, HTML

Changing from batch to streaming in Apache Beam is quite easy, you only need to specify this option or add the flag `--streaming` if launching it from a terminal. 

In [ ]:
# The project will be used for creating a subscription to the Pub/Sub topic and for the Dataflow pipeline
project = google.auth.default()[1]

options = pipeline_options.PipelineOptions(
    streaming=True,
    project=project
)

Since the pipeline uses an unbounded source (Pub/Sub), the next cell sets the maximum duration you want to record the source for replayability without exhausting resources because the data is indefinite.

In [ ]:
ib.options.recording_duration = '1m'

You are ready to launch a pipeline using `InteractiveRunner`. Whenever you want to stop it, you'd need to press the *Stop* on the top left of the notebook or press *i, i*.

In [ ]:
p = beam.Pipeline(InteractiveRunner(), options=options)

topic = "projects/pubsub-public-data/topics/taxirides-realtime"

pubsub = (p | "Read Topic" >> ReadFromPubSub(topic=topic)
            | beam.Map(json.loads))

ib.show(pubsub)

_____________________
Until now we have been run been running the code using runner `InteractiveRunner`, meaning the pipelines are executed in this very notebook. The next pipeline will be executed in Dataflow using `DataflowRunner`.

Since the pipeline is going to run outside the notebook, let's change the level of logging to get more visibility.

In [ ]:
logging.getLogger().setLevel(logging.INFO)

You are also going to need some resources from Google Cloud. The next cell creates a Cloud Storage bucket, a BigQuery Dataset and a Pub/Sub topic.

In [ ]:
!gsutil mb  gs://beam-basics-{project}

!bq mk --location US --dataset beam_basics 
    
!gcloud pubsub topics create beambasics

You need to specify some more options that Dataflow requires, as `temp_location` or `region`.

In [ ]:
def streaming_pipeline(project, region="us-central1"):
    
    topic = "projects/{}/topics/beambasics".format(project)
    table = "{}:beam_basics.from_pubsub".format(project)
    schema = "name:string,score:integer,timestamp:timestamp"
    bucket = "gs://beam-basics-{}".format(project)
    
    options = PipelineOptions(
        streaming=True,
        project=project,
        region=region,
        staging_location="%s/staging" % bucket,
        temp_location="%s/temp" % bucket
    )

    p = beam.Pipeline(DataflowRunner(), options=options)

    pubsub = (p | "Read Topic" >> ReadFromPubSub(topic=topic)
                | "To Dict" >> beam.Map(json.loads)) # Example message: {"name": "carlos", 'score': 10, 'timestamp': "2020-03-14 17:29:00.00000"}

    pubsub | "Write To BigQuery" >> WriteToBigQuery(table=table, schema=schema,
                                  create_disposition=BigQueryDisposition.CREATE_IF_NEEDED,
                                  write_disposition=BigQueryDisposition.WRITE_APPEND)

    return p.run()

In [ ]:
try:
    pipeline = streaming_pipeline(project)
    print("\n PIPELINE RUNNING \n")
except (KeyboardInterrupt, SystemExit):
    raise
except:
    print("\n PIPELINE FAILED")
    traceback.print_exc()

Now there's a running streaming pipeline. Since the pipeline reads from Pub/Sub, you need to publish some messages. The publisher is in the `utils.py` file, which is already imported.

As an example, the elements published have this form:

`{'name': 'carlos' (string), 'score': 22(int), 'timestamp':'2020-03-14 17:29:00.00000' (string)}`

In [ ]:
num_messages = 1000
print("\nLet's wait a bit so the workers can start up.\n")
time.sleep(30)
print("Ok, let's start publishing.\n")
try:
    publish_to_topic(num_messages, "beambasics", project, notebook_number=6, time_division=10)
    print("\n PUBLISHING DONE\n")
except (KeyboardInterrupt, SystemExit):
    raise
except:
    print("\n PUBLISHING FAILED")
    traceback.print_exc()

Now go to your Google Cloud project and check the Dataflow job status. You can also check the BigQuery table (it may take some time until data is available).

In [ ]:
url = ("https://console.cloud.google.com/dataflow/jobs/%s/%s?project=%s" %
 (pipeline._job.location, pipeline._job.id,
  pipeline._job.projectId))
display(HTML("Click <a href=\"%s\" target=\"_new\">here</a> for the details of your Dataflow job." % url))

In [ ]:
!bq query --location US --use_legacy_sql=false 'SELECT * FROM `beam_basics.from_pubsub` LIMIT 10'

If you want to publish more messages before stopping the Dataflow job, run the following cell:

In [ ]:
extra_messages = int(input("How many messages you want to publish? ") or 0)
publish_to_topic(extra_messages, "beambasics", project, notebook_number=6, time_division=10)

## Remember to cancel the running pipelines.

In [ ]:
pipeline.cancel()